In [1]:
# Cellule 1 — Imports et configuration de base

import numpy as np
import torch
import gymnasium as gym
import json
import time

from gymnasium import spaces

from sb3_contrib import MaskablePPO
from env.workshop_env import WorkshopEnv

# === Historique des métriques DAgger ===
history = {
    "supervised_loss": [],   # perte moyenne par itération DAgger
    "reward_collect": [],    # reward élève pendant la phase de collecte
    "reward_eval": []        # reward élève en évaluation (7 jours)
}


# Pour rendre les choses un peu reproductibles
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

In [2]:
# Cellule 2 — Politique experte v3 compatible obs=23 variables + normalisation

from env.workshop_env import WorkshopEnv
import numpy as np

def expert_policy(obs: np.ndarray, env: WorkshopEnv) -> int:
    """
    Politique experte v3 améliorée et sécurisée :
    - réagit à l'ampleur des backlogs
    - commande les MP proportionnellement au backlog total (sécurisé)
    - choisit k en fonction de l'urgence (max 5)
    - priorise P2 puis P1
    - termine d'abord les P2_inter
    - ne renvoie jamais une action hors [0, 200] si on l'utilise correctement via expert_policy_masked
    """

    # ============================================================
    # 1) Dé-normalisation des variables d'état
    # ============================================================

    time = float(obs[0]) * float(env.max_time)

    m1_busy = int(round(obs[1]))
    m1_time_left = float(obs[2]) * 100.0

    m2_busy = int(round(obs[3]))
    m2_time_left = float(obs[4]) * 100.0

    stock_raw = float(obs[5]) * float(env.raw_capacity)
    stock_p1 = float(obs[6]) * float(env.raw_capacity)
    stock_p2_inter = float(obs[7]) * float(env.raw_capacity)
    stock_p2 = float(obs[8]) * float(env.raw_capacity)

    next_delivery_cd = float(obs[9]) * 10080.0

    demande_p1 = float(obs[10]) * 1000.0
    demande_p2 = float(obs[11]) * 1000.0
    q_raw_incoming = float(obs[12]) * 1000.0

    m1_free = (m1_busy == 0)
    m2_free = (m2_busy == 0)

    backlog_p1 = max(demande_p1, 0.0)
    backlog_p2 = max(demande_p2, 0.0)
    backlog_total = backlog_p1 + backlog_p2

    # ============================================================
    # 2) Heuristique dynamique pour choisir k selon backlog
    # ============================================================

    def choose_k(backlog, k_max=5):
        """
        Retourne un entier k :
        - 0 si k_max < 1 (pas de prod possible)
        - sinon entre 1 et k_max_int
        """
        k_max_int = int(k_max)
        if k_max_int < 1:
            return 0

        if backlog <= 5:       k = 1
        elif backlog <= 15:    k = 2
        elif backlog <= 30:    k = 3
        elif backlog <= 60:    k = 4
        else:                  k = 5

        k = min(k, k_max_int)
        return max(1, int(k))

    # ============================================================
    # 3) Politique de commande MP (version sécurisée)
    # ============================================================

    target_raw = min(env.raw_capacity, backlog_total + 10.0)
    current_pipeline = max(0.0, stock_raw + q_raw_incoming)

    missing = target_raw - current_pipeline

    if missing > 0:
        # BORNE STRICTE : 1 ≤ k_cmd ≤ 50
        k_cmd = int(max(1, min(50, missing)))
        action_cmd = 149 + k_cmd  # 150 → 199
        # Sécurité supplémentaire
        if 0 <= action_cmd <= 200:
            return action_cmd

    # ============================================================
    # 4) Priorité absolue : terminer STEP2_P2 si M2 libre
    # ============================================================

    if m2_free and stock_p2_inter > 0:
        k2 = choose_k(backlog_p2, k_max=min(5, stock_p2_inter))
        if k2 > 0:
            action_p2_step2 = 99 + k2  # 100 → 149
            if 0 <= action_p2_step2 <= 200:
                return action_p2_step2

    # ============================================================
    # 5) Priorité n°2 : produire STEP1_P2 si backlog P2 > 0
    # ============================================================

    if m1_free and backlog_p2 > 0 and stock_raw > 0:
        k1_p2 = choose_k(backlog_p2, k_max=min(5, stock_raw))
        if k1_p2 > 0:
            action_p2_step1 = 49 + k1_p2  # 50 → 99
            if 0 <= action_p2_step1 <= 200:
                return action_p2_step1

    # ============================================================
    # 6) Sinon, produire P1 si backlog P1 > 0
    # ============================================================

    if m1_free and backlog_p1 > 0 and stock_raw > 0:
        k1_p1 = choose_k(backlog_p1, k_max=min(5, stock_raw))
        if k1_p1 > 0:
            action_p1 = k1_p1 - 1         # 0 → 49
            if 0 <= action_p1 <= 200:
                return action_p1

    # ============================================================
    # 7) Sinon WAIT
    # ============================================================

    return 200


In [3]:
# Cellule 3 — Politique experte masquée + fonction d'épisode expert

def expert_policy_masked(env: WorkshopEnv, obs: np.ndarray) -> int:
    """
    Politique experte avec prise en compte du masque d'actions.
    Sécurisée : ne renvoie jamais une action hors [0, 200].
    """
    mask = env.get_action_mask().astype(bool)

    a = expert_policy(obs, env)  # action brute proposée par l'expert

    # Sécurité sur le type et la plage des indices
    if not isinstance(a, (int, np.integer)):
        a = 200
    elif a < 0 or a >= len(mask):
        a = 200

    # Si l'action est invalide selon le masque, on corrige
    if not mask[a]:
        # Priorité : WAIT si autorisé
        if mask[200]:
            return 200
        # Sinon, on prend la première action valide
        valid_actions = np.where(mask)[0]
        return int(valid_actions[0])

    return int(a)


def run_expert_episode(env: WorkshopEnv, max_steps: int = 10080):
    """
    Joue un épisode complet avec l'expert masqué.
    Renvoie :
      - obs_array : (T, 23)  # 23 features normalisées
      - act_array : (T,)
      - total_reward
      - nb_steps
    """
    obs, info = env.reset()
    obs_list = []
    act_list = []
    total_reward = 0.0

    for t in range(max_steps):
        action = expert_policy_masked(env, obs)
        obs_list.append(obs.copy())
        act_list.append(action)

        obs, reward, terminated, truncated, info = env.step(action)
        total_reward += reward

        if terminated or truncated:
            break

    obs_array = np.stack(obs_list, axis=0)
    act_array = np.array(act_list, dtype=np.int64)

    return obs_array, act_array, total_reward, t + 1



In [4]:
# ============================================================
# CELLULE 4 — Générer un dataset initial de 10 épisodes experts
# ============================================================

import numpy as np

N_EXPERT_EPISODES = 100   # Nombre d'épisodes experts à générer

dagger_obs = []
dagger_actions = []
expert_rewards = []

print("=== Génération du dataset initial expert ===")

for ep in range(N_EXPERT_EPISODES):
    print(f"\n--- Episode expert {ep+1}/{N_EXPERT_EPISODES} ---")
    
    env_exp = WorkshopEnv()                 # nouvel environnement frais
    obs_ep, act_ep, rew_ep, length_ep = run_expert_episode(env_exp)

    # Stockage
    dagger_obs.append(obs_ep)
    dagger_actions.append(act_ep)
    expert_rewards.append(rew_ep)

    print(f"  ↳ Longueur épisode : {length_ep} steps")
    print(f"  ↳ Reward expert : {rew_ep:.2f}")

# Fusion des données des 10 épisodes
dagger_obs = np.vstack(dagger_obs)
dagger_actions = np.hstack(dagger_actions)

print("\n=== Dataset expert initial généré ===")
print("Taille dagger_obs :", dagger_obs.shape)
print("Taille dagger_actions :", dagger_actions.shape)
print(f"Reward moyen expert sur 100 épisodes : {np.mean(expert_rewards):.2f}")
print("===============================================")


=== Génération du dataset initial expert ===

--- Episode expert 1/100 ---
  ↳ Longueur épisode : 10080 steps
  ↳ Reward expert : 12530.12

--- Episode expert 2/100 ---
  ↳ Longueur épisode : 10080 steps
  ↳ Reward expert : 12938.30

--- Episode expert 3/100 ---
  ↳ Longueur épisode : 10080 steps
  ↳ Reward expert : 12658.06

--- Episode expert 4/100 ---
  ↳ Longueur épisode : 10080 steps
  ↳ Reward expert : 13957.10

--- Episode expert 5/100 ---
  ↳ Longueur épisode : 10080 steps
  ↳ Reward expert : 13475.06

--- Episode expert 6/100 ---
  ↳ Longueur épisode : 10080 steps
  ↳ Reward expert : 13042.08

--- Episode expert 7/100 ---
  ↳ Longueur épisode : 10080 steps
  ↳ Reward expert : 12440.06

--- Episode expert 8/100 ---
  ↳ Longueur épisode : 10080 steps
  ↳ Reward expert : 12760.18

--- Episode expert 9/100 ---
  ↳ Longueur épisode : 10080 steps
  ↳ Reward expert : 13147.98

--- Episode expert 10/100 ---
  ↳ Longueur épisode : 10080 steps
  ↳ Reward expert : 13139.46

--- Episode e

In [5]:
# Cellule 5 — Environnement avec ActionMasker + Modèle MaskablePPO (CPU)

import gymnasium as gym
from sb3_contrib.common.wrappers import ActionMasker

# 1) Fonction de masquage (appelée automatiquement par ActionMasker)
def mask_fn(env):
    return env.get_action_mask()

# 2) Environnement enveloppé
env_student = ActionMasker(WorkshopEnv(), mask_fn)

# 3) Modèle MaskablePPO — FORCÉ SUR CPU
from sb3_contrib import MaskablePPO

model_student = MaskablePPO(
    policy="MlpPolicy",
    env=env_student,
    verbose=1,
    device="cpu",              # <<< ICI : CPU, plus "cuda"
    learning_rate=3e-4,
    gamma=0.99,
    gae_lambda=0.95,
    n_steps=4096,
    batch_size=512,
    clip_range=0.2,
    ent_coef=0.01,
    max_grad_norm=0.5,
    tensorboard_log="./tb_dagger_hybrid"
)

print("Modèle élève initialisé (MaskablePPO + CPU + ActionMasker OK).")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Modèle élève initialisé (MaskablePPO + CPU + ActionMasker OK).


In [6]:
#  CELLULE 6 — Entraînement supervisé DAgger (VERSION LÉGÈRE) 

import torch
import torch.nn as nn
import torch.optim as optim

def train_student_supervised(model, obs_array, act_array, epochs=5, batch_size=256):
    """
    Entraînement supervisé du student :
    Il apprend à imiter l'expert sur le buffer DAgger.

    obs_array : (N, 23)
    act_array : (N,)

    VERSION LÉGÈRE :
    - on limite le nombre d'exemples utilisés à MAX_TRAIN_SAMPLES
      pour éviter de saturer le CPU / la RAM.
    """

    # 0) Limiter la taille du dataset pour rester "safe"
    MAX_TRAIN_SAMPLES = 12000  # à ajuster, mais déjà bien plus léger que 30000

    N_total = len(obs_array)
    if N_total > MAX_TRAIN_SAMPLES:
        # On tire un sous-échantillon aléatoire sans remplacement
        idx = np.random.choice(N_total, size=MAX_TRAIN_SAMPLES, replace=False)
        obs_array = obs_array[idx]
        act_array = act_array[idx]
        print(f"  → Sous-échantillonnage du buffer DAgger : {N_total} → {MAX_TRAIN_SAMPLES} exemples")
    else:
        print(f"  → Utilisation de tous les exemples : {N_total}")

    # 1) Le modèle MaskablePPO tourne en CPU → on aligne l'entraînement dessus
    device = model.policy.device
    policy = model.policy.to(device)

    # 2) Conversion en tenseurs
    X = torch.tensor(obs_array, dtype=torch.float32, device=device)
    y = torch.tensor(act_array, dtype=torch.long, device=device)

    # 3) Sécurité : les actions doivent être dans [0, action_space.n]
    max_action = policy.action_space.n - 1
    y = torch.clamp(y, 0, max_action)

    # 4) Optimiseur + fonction de perte
    optimizer = optim.Adam(policy.parameters(), lr=1e-4)
    loss_fn = nn.CrossEntropyLoss()

    N = len(X)
    nb_batches = (N + batch_size - 1) // batch_size

    print(f"  → Entraînement supervisé sur {N} exemples ({nb_batches} batches)")

    avg_epoch_loss = 0.0

    # 5) Entraînement sur plusieurs epochs
    for epoch in range(epochs):
        perm = torch.randperm(N, device=device)
        Xb = X[perm]
        yb = y[perm]

        total_loss = 0.0

        for i in range(nb_batches):
            start = i * batch_size
            end = min(start + batch_size, N)

            xb_i = Xb[start:end]
            yb_i = yb[start:end]

            # 6) Forward pass (architecture PPO)
            features = policy.extract_features(xb_i)             # encode l'état
            pi_latent, _ = policy.mlp_extractor(features)        # réseau intermédiaire
            logits = policy.action_net(pi_latent)                # logits actions

            loss = loss_fn(logits, yb_i)

            # 7) Backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        epoch_loss = total_loss / nb_batches
        avg_epoch_loss = epoch_loss

        print(f"    Epoch {epoch+1}/{epochs} — loss = {epoch_loss:.4f}")

    print("  ✓ Entraînement supervisé terminé.\n")
    return avg_epoch_loss


In [7]:
# Cellule 7 — Test de l'élève sur une semaine (version MlpPolicy)

def run_student_episode(env, model, max_steps: int = 10080):
    obs, info = env.reset()
    total_reward = 0.0

    for t in range(max_steps):
        # On donne simplement l'observation brute au modèle
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)
        total_reward += reward

        if terminated or truncated:
            break

    return total_reward, t + 1


In [8]:
# ============================================================
# CELLULE 8 — Collecte des données DAgger 
# ============================================================

def collect_dagger_data_from_student(model, max_steps=10080):
    """
    L'élève joue un épisode complet.
    L'expert corrige chaque action.
    On renvoie :
    - obs_list  : toutes les observations rencontrées
    - act_list  : actions expertes correspondantes
    - total_reward_student : reward cumulé du student
    - nb_steps_student     : nombre de steps joués
    """

    env = WorkshopEnv()
    obs, info = env.reset()

    obs_list = []
    act_list = []

    total_reward = 0.0

    for t in range(max_steps):

        #  Très important : récupérer le MASQUE
        mask = env.get_action_mask()

        #  predict() DOIT recevoir action_masks pour éviter actions hors borne
        action_student, _ = model.predict(
            obs,
            deterministic=True,
            action_masks=mask
        )

        # L'expert corrige l'action
        action_expert = expert_policy_masked(env, obs)

        obs_list.append(obs.copy())
        act_list.append(action_expert)

        obs, reward, terminated, truncated, info = env.step(action_student)
        total_reward += reward

        if terminated or truncated:
            break

    return (
        np.array(obs_list, dtype=np.float32),
        np.array(act_list, dtype=np.int64),
        float(total_reward),
        t + 1
    )


In [9]:
# Cellule 9 — Une itération DAgger imitation-seule (SANS PPO)

def dagger_hybrid_iteration(
    model,
    dagger_obs,
    dagger_actions,
    supervised_epochs: int = 3,
    rl_timesteps: int = 10_000
):
    """
    VERSION DIAGNOSTIC : imitation supervisée SEULE
    (on désactive PPO pour voir si le modèle apprend réellement les actions expertes).
    Cette version enregistre les métriques dans le dict global `history`.
    """

    print("\n===== Phase 1 : Imitation supervisée sur buffer DAgger =====")
    loss_value = train_student_supervised(model, dagger_obs, dagger_actions, epochs=supervised_epochs)
    history["supervised_loss"].append(loss_value)

    print("\n===== Phase 2 : (désactivée) =====")
    print("⚠ PPO désactivé volontairement pour test de diagnostic.")

    print("\n===== Phase 3 : Collecte DAgger (élève + expert) =====")
    new_obs, new_actions, R_student, steps_student = collect_dagger_data_from_student(model)
    history["reward_collect"].append(R_student)

    print(f"Reward élève pendant collecte DAgger : {R_student:.2f} sur {steps_student} steps")

    # Agrégation au buffer
    dagger_obs = np.vstack([dagger_obs, new_obs])
    dagger_actions = np.concatenate([dagger_actions, new_actions])

    print("Taille du buffer DAgger après agrégation :", dagger_obs.shape)

    return dagger_obs, dagger_actions


In [10]:
# ======================= CELLULE 10 — DAgger PUR (10080 STEPS) =======================

start_time = time.time()

# Hyperparamètres
N_ITERS            = 80         # itérations DAgger 
SUPERVISED_EPOCHS  = 3
BATCH_SIZE         = 512
EPISODE_STEPS      = 10080     # 1 semaine = 10080 steps
MAX_BUFFER         = 30_000    # borne RAM (à ajuster selon ta machine)

history = {
    "supervised_loss": [],
    "reward_collect": [],
    "reward_eval": []
}

print("\n=== BOUCLE DAgger — VERSION PURE (Épisodes complets 10080 steps) ===\n")


for it in range(1, N_ITERS + 1):

    print(f"\n=========== Iteration {it}/{N_ITERS} DAgger ===========")


    # ----------------------------------------------------------
    # PHASE 1 — Imitation supervisée sur le buffer actuel
    # ----------------------------------------------------------
    print("\n[Phase 1] Entraînement supervisé (imitation expert)")

    avg_loss = train_student_supervised(
        model_student,
        dagger_obs,
        dagger_actions,
        epochs=SUPERVISED_EPOCHS,
        batch_size=BATCH_SIZE
    )

    history["supervised_loss"].append(avg_loss)
    print(f"  → Loss moyenne = {avg_loss:.4f}")


    # ----------------------------------------------------------
    # PHASE 2 — Collecte d'une SEMAINE complète avec l'élève
    #            + corrections de l'expert (DAgger pur)
    # ----------------------------------------------------------
    print("\n[Phase 2] Collecte DAgger (1 semaine simulée)")

    new_obs, new_actions, R_collect, steps = collect_dagger_data_from_student(
        model_student,
        max_steps=EPISODE_STEPS
    )

    print(f"  → Reward élève sur 10080 steps : {R_collect:.2f}")
    print(f"  → Steps joués : {steps}")

    history["reward_collect"].append(R_collect)

    # Ajout au buffer
    dagger_obs = np.vstack([dagger_obs, new_obs])
    dagger_actions = np.hstack([dagger_actions, new_actions])

    # Sécurité mémoire : on ne garde que les MAX_BUFFER derniers exemples
    if dagger_obs.shape[0] > MAX_BUFFER:
        dagger_obs = dagger_obs[-MAX_BUFFER:]
        dagger_actions = dagger_actions[-MAX_BUFFER:]

    print("  → Taille du buffer DAgger :", dagger_obs.shape)


    # ----------------------------------------------------------
    # PHASE 3 — Évaluation de l'élève sur 1 semaine complète
    # ----------------------------------------------------------
    print("\n[Phase 3] Évaluation (1 semaine complète, 10080 steps)")

    env_eval = WorkshopEnv()
    obs, info = env_eval.reset()
    R_eval = 0

    for _ in range(EPISODE_STEPS):

        mask = env_eval.get_action_mask()
        action, _ = model_student.predict(obs, deterministic=True, action_masks=mask)

        obs, r, terminated, truncated, _ = env_eval.step(action)
        R_eval += r

        if terminated or truncated:
            break

    history["reward_eval"].append(R_eval)

    print(f"  → Reward élève en évaluation (10080 steps) : {R_eval:.2f}")
   
    # Sauvegarde de l'historique à chaque itération (sécurisé)
    with open("history_dagger.json", "w") as f:
        json.dump(history, f)

    print(f"✓ Historique partiel sauvegardé (itération {it})")

    # Sauvegarde du modèle élève pour cette itération
    model_student.save(f"student_dagger_iter_{it}.zip")

    # Sauvegarde du dataset toutes les 5 itérations
    if it % 5 == 0:
        np.save(f"dagger_obs_iter_{it}.npy", dagger_obs)
        np.save(f"dagger_actions_iter_{it}.npy", dagger_actions)
        print(f"✓ Dataset sauvegardé (itération {it})")

    # Chrono simple
    elapsed = time.time() - start_time
    print(f" Temps écoulé : {elapsed/60:.1f} min")



print("\n=== FIN DU DAgger — VERSION ÉPISODES 10080 STEPS ===\n")
# Sauvegarde finale une fois le DAgger terminé
model_student.save("student_dagger_final.zip")
print("\n✓ Modèle final sauvegardé : student_dagger_final.zip")



=== BOUCLE DAgger — VERSION PURE (Épisodes complets 10080 steps) ===


=========== Iteration 1/80 DAgger ===========

[Phase 1] Entraînement supervisé (imitation expert)
  → Sous-échantillonnage du buffer DAgger : 1008000 → 12000 exemples
  → Entraînement supervisé sur 12000 exemples (24 batches)
    Epoch 1/3 — loss = 5.2554
    Epoch 2/3 — loss = 5.1390
    Epoch 3/3 — loss = 4.9769
  ✓ Entraînement supervisé terminé.

  → Loss moyenne = 4.9769

[Phase 2] Collecte DAgger (1 semaine simulée)
  → Reward élève sur 10080 steps : -6053.44
  → Steps joués : 10080
  → Taille du buffer DAgger : (30000, 23)

[Phase 3] Évaluation (1 semaine complète, 10080 steps)
  → Reward élève en évaluation (10080 steps) : -5852.28
✓ Historique partiel sauvegardé (itération 1)
 Temps écoulé : 0.5 min

=========== Iteration 2/80 DAgger ===========

[Phase 1] Entraînement supervisé (imitation expert)
  → Sous-échantillonnage du buffer DAgger : 30000 → 12000 exemples
  → Entraînement supervisé sur 12000 exemp